### AP-HP Data exploration

Dans ce notebook, nous analysons les données issues du GH Paris Saclay pour remonter de possibles erreurs ou incohérences dans les données.

In [1]:
import pandas as pd

# deactivate warnings for chaining 
pd.options.mode.chained_assignment = None

In [2]:
# Constants 

code_hospital = {
  '009': 'BRK',
  '010': 'BCT',
  '014': 'APR',
  '028': 'ABC',
  '068': 'RPC',
  '079': 'SPR',
  '096': 'PBR',
}

hospital_name = {
  'BRK': 'BERCK',
  'BCT': 'BICETRE',
  'PBR': 'PAUL-BROUSSE',
  'ABC': 'ANTOINE BECLERE',
  'SPR': 'SAINTE PERINE',
  'APR': 'AMBROISE PARE',
  'RPC': 'RAYMOND POINCARE',
}


In [39]:
# import excel files 

orbis = pd.read_csv('data/valide/orbis.csv', sep=';', encoding='cp1252')
pacs = pd.read_excel('data/valide/pacs.xlsx')
glims = pd.read_excel('data/valide/glims.xlsx')
capacitaire = pd.read_excel('data/valide/capacitaire.xlsx')
sirius = pd.read_excel('data/valide/sirius.xlsx')

## Data Cleaning

In [6]:
# Orbis - take only relevant columns, rename and extract chambre code

orbis = orbis[['IPP', 'Chambre', 'U.Responsabilité']]
orbis.rename(columns={'IPP':'ipp', 'Chambre':'room'}, 
             inplace=True)

# Extract code chambre from chambre (e.g: C134 from C134 - CHAMBRE SEULE C134)
orbis['code_room'] = orbis['room'].str.split(r"\ - ", expand=True)[0]

# Extract hospital name from U.Responsabilité (e.g: ABC from 028081 - ABC OBSTETRIQUE (UF))
orbis['hospital_name'] = orbis['U.Responsabilité'].str.split(r"\ - ", expand=True)[1].str[0:3]

# From room, remove white spaces and uppercase
orbis['room'] = orbis['room'].str.replace(' ', '').str.upper()

orbis = orbis[['ipp', 'code_room', 'room', 'hospital_name']]

In [7]:
# Glims - rename columns, drop_duplicates, map is_pcr value to 1

glims = glims[['ipp', 'is_pcr']]
glims.drop_duplicates('ipp', inplace=True) #note: drop_duplicate only returns first value for each IPP
glims['is_pcr'] = glims['is_pcr'].map({'Positif':1})

In [8]:
# Pacs - select columns, drop_duplicates
pacs = pacs[['ipp', 'radio']]
pacs.drop_duplicates('ipp', inplace=True)

In [9]:
# Capacitaire, rename is_covid column
capacitaire.rename(columns={'Full COVID 1/0':'is_covid_dedicated',
                            'hopital':'hospital_name',
                            'service_covid':'covid_service'}, inplace=True)

# Capacitaire, rename is_covid column
capacitaire.drop_duplicates(['hospital_name', 'covid_service'], inplace=True)

In [10]:
# Sirius, select columns, rename and filter 
sirius = sirius[['Hopital', 
                 'Intitulé Site Crise COVID', 
                 'Retenir ligne O/N',
                 'Libelle Chambre',
                 'Code Chambre']]

sirius.rename(columns={'Hopital':'code_hospital',
                       'Intitulé Site Crise COVID': 'covid_service',
                       'Retenir ligne O/N':'filter_row',
                       'Code Chambre':'code_room',
                       'Libelle Chambre':'label_room'},
             inplace=True)

# only filter row with OUI 
sirius = sirius.query("filter_row=='OUI'")

# remove white spaces from label room, uppercase 
sirius['label_room'] = sirius['label_room'].astype(str)
sirius['label_room'] = sirius['label_room'].str.replace(' ', '').str.upper()

# remove white spaces from code room, uppercase 
sirius['code_room'] = sirius['code_room'].astype(str)
sirius['code_room'] = sirius['code_room'].str.replace(' ', '').str.upper()

#create room as concatenate of code_room and label_room

sirius['room'] = sirius[['code_room', 'label_room']].agg('-'.join, axis=1)

sirius.drop('filter_row', axis=1, inplace=True)

## Merge

In [11]:
# merge Orbis with Glims

orbis =\
    pd.merge(orbis,
             glims,
             on='ipp',
             how='left')

# merge with Pacs

orbis =\
    pd.merge(orbis,
             pacs,
             on='ipp',
             how='left')

orbis.fillna(0, inplace=True)

In [12]:
# Compute is_covid patient

def is_covid(x):
    if (x['is_pcr'] + x['radio']) > 0:
        return 1
    else:
        return 0

orbis['is_covid'] = orbis.apply(is_covid, axis=1)

In [13]:
# How many Covid patients do not have a room associated with ? 
print('il y a {n_patients_no_room} patients sans'
       ' Chambre dans Orbis dont {n_patients_no_room_covid} Covid positif:'\
      .format(n_patients_no_room = len(orbis.query("code_room == ''")),
              n_patients_no_room_covid = len(orbis.query("code_room == ''").query("is_covid==1")),
             ))

orbis.query("code_room == ''").query("is_covid==1")

il y a 98 patients sans Chambre dans Orbis dont 8 Covid positif:


,ipp,code_room,room,hospital_name,is_pcr,radio,is_covid
118,8003232267,,-,ABC,1.0,0.0,1
381,8014201282,,-,APR,1.0,0.0,1
402,8001043127,,-,APR,1.0,0.0,1
760,8014207211,,-,BCT,1.0,0.0,1
780,8014213145,,-,BCT,1.0,0.0,1
821,8008578325,,-,BCT,1.0,0.0,1
1182,8014232794,,-,BCT,1.0,0.0,1
1301,8008890155,,-,BCT,1.0,0.0,1


In [14]:
# add hospital_name to sirius 

sirius['hospital_name'] = sirius['code_hospital'].map(code_hospital)

In [15]:
# merge Sirius and Orbis

orbis =\
    pd.merge(orbis,
             sirius,
             how='left',
             on=['hospital_name', 
                 'room'])

### Discrepancies between Sirius and Orbis room matching

After a left merge on the room matching key, we end up with rooms that do not match in Sirius. 

In [16]:
print('il y a {n_rooms_missing} chambres Orbis'
      ' qu\'on ne retrouve pas dans Sirius'\
      .format(n_rooms_missing=len(orbis.query("code_hospital.isnull()"))))

il y a 147 chambres Orbis qu'on ne retrouve pas dans Sirius


Why is it the case? Let's look at the difference in label in the two files: 

In [17]:
# create a df with the rooms that did not find a match in Sirius

room_gap = orbis[orbis['covid_service'].isna()][['code_room_x', 'room', 'hospital_name']]
room_gap.rename(columns={'code_room_x':'code_room'}, 
                inplace=True)

A first reason is that the code_room in Orbis can be empty. This can be patients that have not yet a room allocated or newborns that sleep with their mama. How many are those? 

In [18]:
n_patients_with_no_rooms_in_orbis = len(room_gap.query("code_room == ''"))
n_patients_remaining = len(orbis.query("code_hospital.isnull()")) - n_patients_with_no_rooms_in_orbis

print('il y a {n_patients_with_no_rooms_in_orbis} patients dans Orbis'
      ' sans chambre. Reste a expliquer {n_patients_remaining} patients'\
      .format(n_patients_with_no_rooms_in_orbis=n_patients_with_no_rooms_in_orbis,
              n_patients_remaining=n_patients_remaining))

il y a 98 patients dans Orbis sans chambre. Reste a expliquer 49 patients


In [19]:
# merge room_gap with sirius only on those that have a code_room in Sirius

df_gap = pd.merge(room_gap.query("code_room != ''"),
         sirius,
         on=['code_room', 'hospital_name'],
         how='left')

Another reason is that the code_room does not match. This can be because when imported, sirius Excel file cast columns and consider some rows as string or integer. See below an example for code_room '04' in Orbis and '4' in Sirius: 

In [20]:
df_gap.query("code_room=='04'")

,code_room,room_x,hospital_name,code_hospital,covid_service,label_room,room_y


In [21]:
sirius.query("code_room=='4'").query("hospital_name=='BCT'")

,code_hospital,covid_service,label_room,code_room,room,hospital_name


Note that it is not as straightforward as adding one trailing 0. Some room code have two trailing zeros for instance: 

In [22]:
df_gap.query("code_room=='003'")

,code_room,room_x,hospital_name,code_hospital,covid_service,label_room,room_y


In [23]:
sirius.query("code_room=='3'").query("hospital_name=='RPC'")

,code_hospital,covid_service,label_room,code_room,room,hospital_name


Other does not seem to match at all: 

In [24]:
df_gap.query("code_room == 'L205'")

,code_room,room_x,hospital_name,code_hospital,covid_service,label_room,room_y
18,L205,L205-CHAMBRE5LAENNEC0205,BCT,NaN,NaN,NaN,NaN


In [25]:
sirius.query("hospital_name == 'RPC'").query('label_room.str.contains("205")')

,code_hospital,covid_service,label_room,code_room,room,hospital_name
1412,68,Widal 2 Covid+,CHAMBRE205,205,205-CHAMBRE205,RPC


In [26]:
n_patients_with_no_code_room_matching_in_sirius =\
    len(df_gap.drop_duplicates(['room_x', 
                                'hospital_name']).query('code_hospital.isnull()'))
n_patients_remaining = n_patients_remaining - n_patients_with_no_code_room_matching_in_sirius

print('il y a {n_patients_with_no_code_room_matching_in_sirius} patients dont le code chambre'
      ' Sirius ne correspond pas dans Orbis. Reste a expliquer {n_patients_remaining} patients'\
      .format(n_patients_with_no_code_room_matching_in_sirius=n_patients_with_no_code_room_matching_in_sirius,
              n_patients_remaining=n_patients_remaining))

il y a 5 patients dont le code chambre Sirius ne correspond pas dans Orbis. Reste a expliquer 44 patients


Finally, we have room labels that do not match because strings are different. A few examples: 

- There is a space at the end "ABC" vs "ABC "
- Lower case vs Upper case "CHAMBRE 5 NEO NATALE" vs "Chambre 5 Neo Natale"
- Characters used 321 - "ferm√©" vs "ferm?"

In [27]:
df_gap.query("code_hospital.notnull()")[['room_x', 
                                         'room_y']]\
      .rename(columns={'room_x':'room_orbis',
                       'room_y':'room_sirius'})

,room_orbis,room_sirius
1,CHA1-CHAMBRESRMN.A1,CHA1-CHAMBRESRMN°A1
2,P14-CHAMBRE14PÉDIATRIE,P14-CHAMBRE14P�DIATRIE
3,CHA5-CHAMBRESRMN.A5,CHA5-CHAMBRESRMN°A5
4,CHD4-CHAMBRESCMN.D4,CHD4-CHAMBRESCMN°D4
5,CHD2-CHAMBRESCMN.D2,CHD2-CHAMBRESCMN°D2
6,P02-CHAMBRE2PÉDIATRIE,P02-CHAMBRE2P�DIATRIE
7,CHE2-CHAMBRESCMN.E2,CHE2-CHAMBRESCMN°E2
8,CHB4-CHAMBRESRMN.B4,CHB4-CHAMBRESRMN°B4
9,P01-CHAMBRE1PÉDIATRIE,P01-CHAMBRE1P�DIATRIE
10,CHA2-CHAMBRESRMN.A2,CHA2-CHAMBRESRMN°A2


In [28]:
n_patients_with_label_diff =\
    len(df_gap.query("code_hospital.notnull()")\
              .drop_duplicates(['room_x', 
                                'hospital_name']))
n_patients_remaining = n_patients_remaining - n_patients_with_label_diff

print('il y a {n_patients_with_label_diff} patients avec une difference de label'
      ' . Reste a expliquer {n_patients_remaining} patients'\
      .format(n_patients_with_label_diff=n_patients_with_label_diff,
              n_patients_remaining=n_patients_remaining))

il y a 40 patients avec une difference de label . Reste a expliquer 4 patients


## Capacitaire

In [29]:
# merge capacitaire 

orbis =\
    pd.merge(orbis,
             capacitaire,
             how='left',
             on=['hospital_name', 
                 'covid_service'])

In [30]:
df = orbis.groupby(['hospital_name', 
               'covid_service'])\
     .agg({'ipp': 'nunique',
           'is_covid': 'sum',
           'lits_ouverts': 'max',
           'lits_ouverts_covid': 'max'})\
     .reset_index()\
     .fillna(0)

In [59]:
df.head()

,hospital_name,covid_service,ipp,is_covid,lits_ouverts,lits_ouverts_covid
0,ABC,COVID 14,11,7,14.0,14.0
1,ABC,COVID 30 (ex chir viscéral et digestive),27,19,0.0,0.0
2,ABC,HGE,24,0,27.0,0.0
3,ABC,MAP COVID + ET COVID -,23,16,0.0,0.0
4,ABC,MATERNITE,84,4,0.0,0.0
